In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import csv
import sys
import flwr as fl
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import random

splitDataset

In [2]:
def load_data(filename):
    df = pd.read_csv(filename)
    x, y = df.iloc[:, :-1], df.iloc[:, [-1]].values.ravel()
    labelencoder = LabelEncoder()
    classe_encoder = labelencoder.fit_transform(y)
    classe_dummy = np_utils.to_categorical(classe_encoder)
    return x, y, classe_dummy

def split_data(filename, numero_linhas):
    with open(filename, 'r') as file:
        # empty dictionary
        row_dict = {}

        reader = csv.reader(file)
        #header = next(reader)
        n_linhas = 0
        itt = 0
        new_dataset_rows = []
        shuffle_dataset_rows = []
        for row in reader:
            print("ROW: ", row)
            row_dict[row[-1]] = []
            shuffle_dataset_rows.append(row)
            #n_linhas = n_linhas + 1

        random.shuffle(shuffle_dataset_rows)
        for row in shuffle_dataset_rows:
            new_dataset_rows.append(row)
            n_linhas = n_linhas + 1
            if n_linhas >= numero_linhas:
                with open(str(itt) + '_dataset_500.csv', 'w', newline='') as f:
                    writer = csv.writer(f)
                    writer.writerows(new_dataset_rows)
                n_linhas = 0
                new_dataset_rows = []
                itt = itt + 1

    #print("Numero de Classes: ", len(row_dict.keys()))
    #print("Numero de Linhas: ", n_linhas)


#split_data("500_2018_challenge.csv", 1500)

BASE_PATH = "/home/joaoneto/biometria/csv_files"

MIN_NUM_ROWS = 50000
MAX_NUM_ROWS = 100000
NUM_FRAMES = 250

users_statistics = load_data("/home/joaoneto/fl_projects/fl_behavior/scomnet/users_statistics.csv")

valid_users = users_statistics[(users_statistics.nrows >= MIN_NUM_ROWS) & (users_statistics.nrows <= MAX_NUM_ROWS)]["player_id"].unique()

tmp_data = []
for user in valid_users:
    for csv_file_path in glob.glob(f"{BASE_PATH}/{user}/*.csv"):
        tmp_data.append(pd.read_csv(csv_file_path))

data = pd.concat(tmp_data)
data.reset_index(inplace=True, drop=True)

users = data['player_id'].unique()
    
train_set, user_list = split_data(data, users, NUM_FRAMES)
train_set = np.array([np.array(x) for x in train_set]) 
train_set_join = train_set.reshape(train_set.shape[0], 384)
data_join = pd.DataFrame(train_set_join)
data_join['user'] = user_list

x, y, y_dumm = load_data(data_join)

#x, y, y_dumm = load_data("100_2018_challenge.csv")
#print("y_dumm: ", y_dumm)
"""
for a, b in zip(y, y_dumm):
    print(a," ",b)
"""


FileNotFoundError: [Errno 2] No such file or directory: '/home/joaoneto/biometria/csv_files/users_statistics.csv'

dataset_info

In [ ]:
def load_data(filename):
    df = pd.read_csv(filename)
    x, y = df.iloc[:, :-1], df.iloc[:, [-1]].values.ravel()
    return df

df = load_data("2018_challenge_cleaned.csv")
df['person'].value_counts().plot()
#plt.show()

df2 = df.dropna()
counts = df['person'].value_counts()
print(len(df.person.value_counts()))

df2 = df[~df['person'].isin(counts[counts < 120].index)]
print(len(df2.person.value_counts()))
df2.to_csv('less_2018_challenge.csv', index=False)

with open('less_2018_challenge.csv', 'r') as file:
    reader = csv.reader(file)
    n_linhas = 0
    header = next(reader)
    row_dict = {}
    if header != None:
        for row in reader:
            #if n_linhas < 10:
            #    print("ROW: ", row)
            #    print("ROW: ", row[-1])
            if row[-1] not in row_dict:
                row_dict[row[-1]] = []
                row_dict[row[-1]].append(row)
            else:
                row_dict[row[-1]].append(row)
            n_linhas = n_linhas + 1

#print("row_dict.keys(): ", row_dict.keys())
count = 0

dataset_100 = []
dataset_200 = []
dataset_500 = []

for chave in row_dict.keys():
    print("chave: ", chave)
    print("count: ", count)
    for linha in row_dict[chave]:
        if count < 100:
            dataset_100.append(linha)
        if count < 200:
            dataset_200.append(linha)
        if count < 500:
            dataset_500.append(linha)
    count = count + 1
    if count > 500:
        break

with open('100_2018_challenge.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(dataset_100)

with open('200_2018_challenge.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(dataset_200)

with open('500_2018_challenge.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(dataset_500)



Cliente

In [ ]:
data_loss_training = []
data_loss_test = []
data_acc_training = []
data_acc_teste = []

def load_data(filename):
    df = pd.read_csv(filename)
    x, y = df.iloc[:, :-1], df.iloc[:, [-1]].values.ravel()
    return x, y

def convertStrToListFloat(stringList):
    final_list_float_numpy = []
    for lista in stringList:
        lista_replace = lista.replace("[","").replace("]","")
        lista_float = [float(s) for s in lista_replace.split(',')]
        final_list_float_numpy.append(lista_float)
    return numpy.asarray(final_list_float_numpy)

# Define Flower client
class CifarClient(fl.client.NumPyClient):
    def get_parameters(self):
        return model.get_weights()

    def fit(self, parameters, config):
        #Carregar o modelo da memoria
        lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-3 * 10 ** (epoch / 20))
        model.set_weights(parameters)
        #model.fit(x_train, y_train, epochs=1, batch_size=26, steps_per_epoch=1)
        dados_resultados = model.fit(x_train, y_train, epochs=1, callbacks=[lr_scheduler])
        print("dados_resultados: ", dados_resultados.history['loss'])
        print("dados_resultados: ", dados_resultados.history['accuracy'])
        data_loss_training.append(dados_resultados.history['loss'])
        data_acc_training.append(dados_resultados.history['accuracy'])
        # SALVAR o modelo da memoria
        return model.get_weights(), len(x_train), {}

    def evaluate(self, parameters, config):
        # CARREGAR o modelo da memoria
        model.set_weights(parameters)
        loss, accuracy = model.evaluate(x_test, y_test)
        print("LOSS: ", loss)
        print("ACCURACY: ", accuracy)
        data_loss_test.append([loss])
        data_acc_teste.append([accuracy])
        # SALVAR o modelo da memoria
        return loss, len(x_test), {"accuracy": accuracy}


if __name__ == "__main__":
    # Load and compile Keras model
    #model = tf.keras.applications.MobileNetV2((32, 32, 1971), classes=1971, weights=None)  # Cada coluna, 32, Quantidade de Classes, Quantidade de Classes
    #model = tf.keras.applications.MobileNetV2((32, 32, 1971), classes=1971, weights=None)  # Cada coluna, 32, Quantidade de Classes, Quantidade de Classes

    model = tf.keras.Sequential([
        tf.keras.Input(shape=(26,)),
        tf.keras.layers.Dense(12, activation="relu"),
        tf.keras.layers.Dense(12, activation="relu"),
        tf.keras.layers.Dense(12, activation="relu"),
        tf.keras.layers.Dense(12, activation="relu"),
        tf.keras.layers.Dense(1971, activation="softmax")
    ])

    model.compile("adam", "categorical_crossentropy", metrics=["accuracy"])

    arquivo = sys.argv[1]
    teste_arquivo = arquivo.split("_")
    teste_arquivo = str(int(teste_arquivo[0])+19)+"_"+teste_arquivo[1]

    # Load dataset
    x_train, y_train = load_data(arquivo)
    y_train = convertStrToListFloat(y_train)

    x_test, y_test = load_data(teste_arquivo)
    y_test = convertStrToListFloat(y_test)

    # Start Flower client
    fl.client.start_numpy_client(server_address="localhost:8080", client=CifarClient())

    #data_loss_training = []
    #data_loss_test = []
    #data_acc_training = []
    #data_acc_teste = []

    with open(arquivo.replace("_dataset.csv","") + '_resultado_loss_training.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(data_loss_training)

    with open(arquivo.replace("_dataset.csv","") + '_resultado_loss_teste.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(data_loss_test)

    with open(arquivo.replace("_dataset.csv","") + '_resultado_acc_training.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(data_acc_training)

    with open(arquivo.replace("_dataset.csv","") + '_resultado_acc_teste.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(data_acc_teste)





Servidor

In [ ]:
# Start Flower server for three rounds of federated learning
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.1,  # Sample 10% of available clients for the next round
    min_fit_clients=10,  # Minimum number of clients to be sampled for the next round
    min_available_clients=10,  # Minimum number of clients that need to be connected to the server before a training round can start
)
fl.server.start_server(server_address="localhost:8080", config={"num_rounds": 100}, strategy=strategy) #Aumentar o numero de rounds
